In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from statsmodels.formula.api import ols
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
import matplotlib.pyplot as plt
import statsmodels.api as sm
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
import string
import re

# Notice that these vectorizers are from `sklearn` and not `nltk`!
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer
%matplotlib inline

In [2]:
df= pd.read_csv('CraigslistCars.csv')

In [3]:
df_1 = df.copy()

In [ ]:
df_1.head()


In [ ]:
df_1.sample()

In [ ]:
df_1.columns

In [ ]:
df_1.price.min(), df_1.price.max()

In [ ]:
df_1.loc[df_1['price'] == df_1.price.min(), ['year']], df_1.loc[df_1['price']==df_1.price.max(), ['year']]

In [ ]:
# Want to remove the outliers and the cars with 0 as a value of the data on price, look at an inner 90% of the data

In [ ]:
df_1['price'] = df_1['price'][df_1['price'].between(df_1['price'].quantile(.085), df_1['price'].quantile(.985),inclusive=True)]
df_1.price.min(), df_1.price.max()
# shows 100 - 59999 is an inner 90%%

In [ ]:
# Want new df with just price values that range from 100 to 59999

In [4]:
df_2 = df_1.loc[(df_1['price']>100) & (df_1['price'] <59999)].copy()
df_2.head(), len(df_2)

(           id                                                url  \
 0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
 1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
 2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
 3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
 4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   
 
                    region                         region_url  price  year  \
 0                prescott    https://prescott.craigslist.org   6000   NaN   
 1            fayetteville       https://fayar.craigslist.org  11900   NaN   
 2            florida keys        https://keys.craigslist.org  21000   NaN   
 3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
 4              greensboro  https://greensboro.craigslist.org   4900   NaN   
 
   manufacturer model condition cylinders  ... size  type paint_color  \
 0          NaN   NaN       NaN       NaN

In [26]:
df_2.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1018
manufacturer     15164
model             4321
condition       145292
cylinders       155429
fuel              2564
odometer          2221
title_status      7543
transmission      1775
VIN             148750
drive           117540
size            276055
type             82942
paint_color     113668
image_url           62
description         64
county          383903
state                0
lat               3461
long              3461
posting_date        62
dtype: int64

In [ ]:
# year, manufacturer, model, condition, cylinders, fuel, odometer, title_status, transmission, VIN, drive, size, type, paint_color
# image_url, description, county, state, lat, long, posting_date have Na values

Deal With Na

In [ ]:
len(df_2)

In [ ]:
df_2['county'].value_counts()
# is completely empty

In [5]:
df_2.drop('county', axis=1, inplace=True)
df_2.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


County dealt with

In [ ]:
df_2['year'].value_counts()

In [ ]:
df_2['year'].min()

In [ ]:
df_2.year.max()

In [ ]:
unique_years = df_2['year'].unique()
print(sorted(unique_years))

In [ ]:
type(unique_years)

In [ ]:
df_2.year.value_counts(normalize=True)

In [ ]:
# Since only 1018 cars don't have the year it is ok to just drop the rows without a year as it only pertains to .3% of the
# data


In [6]:
df_2.dropna(subset=['year'], axis =0,inplace=True)

In [29]:
df_2.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     14147
model             4259
condition       144274
cylinders       155361
fuel              2312
odometer          2158
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [ ]:
unique_manufacturer = df_2['manufacturer'].unique()
print(unique_manufacturer)

In [ ]:
# need to find a way to fill in missing manufacturers. too many Na's to drop

In [ ]:
# see if any other columns have the manufacturer name 
# list is in and return true or false

In [ ]:
sample = df_2['description'][97]

In [ ]:
len(sample)

In [ ]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
sample_doc = tokenizer.tokenize(sample)
sample_doc = [token.lower() for token in sample_doc]
sw =stopwords.words('english')
sample_doc = [token for token in sample_doc if token not in sw]

In [ ]:
sample_doc

In [ ]:
len(set(sample_doc))

In [ ]:
print(set(sample_doc))

In [ ]:
type(sample_doc)

In [ ]:
# List1
List1 = unique_manufacturer
 
# List2
List2 = sample_doc

check =  any(item in List1 for item in List2)
 
if check is True:
    print("The list {} contains all elements of the list {}".format(List1, List2))    
else :
    print("No, List1 doesn't have all elements of the List2.")

In [ ]:
def list_contains(List1, List2): 
    check = False
  
    # Iterate in the 1st list 
    for m in List1: 
  
        # Iterate in the 2nd list 
        for n in List2: 
    
            # if there is a match
            if m == n: 
                check = True
                return check  
                  
    return check 
      
# Test Case 1
List1 = unique_manufacturer
List2 = sample_doc 
print("Test Case#1 ", list_contains(List1, List2)) 

In [ ]:
car = set(unique_manufacturer) & set(sample_doc)
car

In [ ]:
# df_2['model']=df_2['model'].str.lower().str.split()

In [ ]:
# df_2['description']=df_2['description'].str.lower()

In [ ]:
# df_2.sample(10)

In [ ]:
# df_2.sample()

In [ ]:
# df_2['exists'] = df.drop('manufacturer', 1).isin(df['manufacturer']).any(1) 
# df_2.exists.value_counts()

In [ ]:
# Only 8 cases where the name of the manufacturer's name is mentioned outside of the manufacturers column therefore wan't
# able to fill in the Na Values

In [16]:
manu_desc=df_2.loc[df_2['manufacturer'].isna(), ['description','manufacturer']].copy()
manu_desc.head()

,description,manufacturer
97,Carvana is the safer way to buy a car During t...,NaN
122,"2005 Blue Bird 12 window, 72 passenger bus tha...",NaN
135,Carvana is the safer way to buy a car During t...,NaN
137,Great Chevy Barn Find Great candidate for res...,NaN
155,Carvana is the safer way to buy a car During t...,NaN


In [17]:
manu_desc.reset_index(inplace=True)
manu_desc.head()

,index,description,manufacturer
0,97,Carvana is the safer way to buy a car During t...,NaN
1,122,"2005 Blue Bird 12 window, 72 passenger bus tha...",NaN
2,135,Carvana is the safer way to buy a car During t...,NaN
3,137,Great Chevy Barn Find Great candidate for res...,NaN
4,155,Carvana is the safer way to buy a car During t...,NaN


In [ ]:
sample2 = manu_desc['description']

In [ ]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
sample_doc = tokenizer.tokenize(sample)
sample_doc = [token.lower() for token in sample_doc]
sw =stopwords.words('english')
sample_doc = [token for token in sample_doc if token not in sw]

In [10]:
pattern_2 = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern_2)

In [11]:

manu_desc['tokenized_sents'] = manu_desc.apply(lambda row: tokenizer.tokenize(row['description']), axis=1)


In [12]:
manu_desc['new'] = manu_desc['tokenized_sents'].str.lower()
manu_desc.head()

,index,description,manufacturer,tokenized_sents,new
0,97,Carvana is the safer way to buy a car During t...,NaN,"[Carvana, is, the, safer, way, to, buy, a, car...",NaN
1,122,"2005 Blue Bird 12 window, 72 passenger bus tha...",NaN,"[Blue, Bird, window, passenger, bus, that, is,...",NaN
2,135,Carvana is the safer way to buy a car During t...,NaN,"[Carvana, is, the, safer, way, to, buy, a, car...",NaN
3,137,Great Chevy Barn Find Great candidate for res...,NaN,"[Great, Chevy, Barn, Find, Great, candidate, f...",NaN
4,155,Carvana is the safer way to buy a car During t...,NaN,"[Carvana, is, the, safer, way, to, buy, a, car...",NaN


In [18]:
manu_desc['description'] =manu_desc['description'].str.split()
manu_desc.head()

,index,description,manufacturer
0,97,"[Carvana, is, the, safer, way, to, buy, a, car...",NaN
1,122,"[2005, Blue, Bird, 12, window,, 72, passenger,...",NaN
2,135,"[Carvana, is, the, safer, way, to, buy, a, car...",NaN
3,137,"[Great, Chevy, Barn, Find, Great, candidate, f...",NaN
4,155,"[Carvana, is, the, safer, way, to, buy, a, car...",NaN


In [19]:
# manu_desc['description'] = manu_desc['description'].lower()
# manu_desc.head()

AttributeError: 'Series' object has no attribute 'lower'

In [ ]:
# manu_desc['description'] = manu_desc['description'].str.lower()

In [ ]:
manu_desc.head()

In [ ]:
manu_desc.head()

In [ ]:
type(manu_desc['tokenized_sents'][0])

In [ ]:
manufacturer= list(unique_manufacturer)
manufacturer

In [ ]:
# manu_desc['exists'] = manu_desc.drop('manufacturer', 1).isin(manu_desc['manufacturer']).any(1) 
# manu_desc.exists.value_counts()

In [ ]:
# manu_desc["test"] = np.where(manu_desc["tokenized_sents"].isin(manufacturer), 'Other')


In [ ]:
manu_desc['tokenized_sents'].lower()

In [ ]:
car1 = set(unique_manufacturer) & set(manu_desc['tokenized_sents'][0])
car1

In [ ]:
# set(manu_desc['tokenized_sents'][0])

In [ ]:
df_new = manu_desc[manu_desc['tokenized_sents'].isin(['toyota'])]
df_new.head()

In [ ]:
empty = []
# for car in unique_manufacturer:
    
#     if car in manu_desc['tokenized_sents'][0]:
manu_desc['tokenized_sents'][0]

In [123]:
# df1['condition'] = df1.apply( lambda x: x['condition'] if pd.isna( x['model'] ) or pd.notna( x['condition'] ) else condition_dict[x['model']] if x['model'] in condition_dict.keys() else x['condition'], axis=1)# 28615 "nan" values left.
# as it can cause confusion between the "like new" and "new" values I will not try to fill in the "nan" values using the ifo's in the "description" variable.


In [8]:
df_condition_completed = df_2.loc[df_2['condition'].notna()].copy()
df_with_model_and_condition = df_condition_completed.loc[df_condition_completed['model'].notna()].copy()
condition_dict = {}
for a, b in df_with_model_and_condition[['model', 'condition']].values:
    condition_dict[a] = b

In [41]:
# df_condition_completed = df_2[df_2['condition'].notna()]
# df_with_model_and_condition = df11[df11['model'].notna()]
# condition_dictionary = {}
# for a, b in df_with_model_and_condition[['model', 'condition']].values:
#     condition_dict[a] = b

In [9]:
df_2['condition'] = df_2.apply(lambda x: x['condition'] if pd.isna(x['model']) or pd.notna(x['condition']) else condition_dict[x['model']] if x['model'] in condition_dict.keys() else x['condition'], axis=1)

In [126]:
df_2.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     14147
model             4259
condition        19900
cylinders       155361
fuel              2312
odometer          2158
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [33]:
df_2['condition'].value_counts()

excellent    158208
good         155156
like new      34081
fair          12740
new            1417
salvage        1383
Name: condition, dtype: int64

In [59]:
df_2['cylinders'].value_counts()

6 cylinders     86907
4 cylinders     72223
8 cylinders     63899
5 cylinders      1662
10 cylinders     1102
other             994
3 cylinders       591
12 cylinders      146
Name: cylinders, dtype: int64

In [10]:
df_cylinders_completed = df_2.loc[df_2['cylinders'].notna()].copy()
df_cylinders_and_model = df_cylinders_completed.loc[df_cylinders_completed['model'].notna()].copy()
cylinders_dictionary = {}
for c, d in df_cylinders_and_model[['model', 'cylinders']].values:
    cylinders_dictionary[c] = d

In [11]:
df_2['cylinders'] = df_2.apply( lambda x: x['cylinders'] if pd.isna( x['model'] ) or pd.notna( x['cylinders'] ) else cylinders_dictionary[x['model']] if x['model'] in cylinders_dictionary.keys() else x['cylinders'], axis=1)

In [12]:
df_2.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     14147
model             4259
condition        19900
cylinders        36992
fuel              2312
odometer          2158
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [65]:
df_2['cylinders'].value_counts()

6 cylinders     110173
4 cylinders      92340
8 cylinders      88659
5 cylinders       2582
3 cylinders       1368
10 cylinders      1182
other             1115
12 cylinders       155
Name: cylinders, dtype: int64

In [78]:
df_2['fuel'].value_counts()

gas         323825
other        27117
diesel       23257
hybrid        4813
electric      1561
Name: fuel, dtype: int64

In [79]:
df_2['title_status'].value_counts()

clean         362567
rebuilt         6976
salvage         3796
lien            1362
missing          753
parts only       172
Name: title_status, dtype: int64

In [80]:
df_2['transmission'].value_counts()

automatic    297276
other         60749
manual        23147
Name: transmission, dtype: int64

In [81]:
df_2['model'].value_counts()

f-150                       6970
silverado 1500              4480
1500                        3719
camry                       2783
accord                      2714
                            ... 
van 30                         1
1968 speciality                1
titan sv crew cab v8 4x4       1
1992 International 4900        1
1976 grand Marquis             1
Name: model, Length: 27161, dtype: int64

In [ ]:
decision 

In [83]:
df_2.loc[df_2['cylinders'].isna(), ['model']].value_counts()

model                        
silverado                        1484
2500                             1168
altima                           1162
wrangler                         1092
escape                           1065
                                 ... 
freightliner m2106                  1
frieghtliner                        1
frightliner m2                      1
frontier crew cab 4x2 sv auto       1
*matrix*                            1
Length: 8526, dtype: int64

In [84]:
df_3 = df_2.copy()

In [85]:
df_3.set_index('model',inplace=True)

In [86]:
df_3.head()

,id,url,region,region_url,price,year,manufacturer,condition,cylinders,fuel,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
model,,,,,,,,,,,,,,,,,,,,,
sierra 1500 crew cab slt,7316814884,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,33590,2014.0,gmc,good,8 cylinders,gas,...,NaN,NaN,pickup,white,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T12:31:18-0500
silverado 1500,7316814758,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,22590,2010.0,chevrolet,good,8 cylinders,gas,...,NaN,NaN,pickup,blue,https://images.craigslist.org/00R0R_lwWjXSEWNa...,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T12:31:08-0500
silverado 1500 crew,7316814989,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,39590,2020.0,chevrolet,good,8 cylinders,gas,...,NaN,NaN,pickup,red,https://images.craigslist.org/01212_jjirIWa0y0...,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T12:31:25-0500
tundra double cab sr,7316743432,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,30990,2017.0,toyota,good,8 cylinders,gas,...,NaN,NaN,pickup,red,https://images.craigslist.org/00x0x_1y9kIOzGCF...,Carvana is the safer way to buy a car During t...,al,32.590,-85.4800,2021-05-04T10:41:31-0500
f-150 xlt,7316356412,https://auburn.craigslist.org/cto/d/auburn-uni...,auburn,https://auburn.craigslist.org,15000,2013.0,ford,excellent,6 cylinders,gas,...,rwd,full-size,truck,black,https://images.craigslist.org/00404_l4loxHvdQe...,2013 F-150 XLT V6 4 Door. Good condition. Leve...,al,32.592,-85.5189,2021-05-03T14:02:03-0500


In [87]:
df_3.sort_values(by=['model'],inplace=True)

In [88]:
df_3.head()

,id,url,region,region_url,price,year,manufacturer,condition,cylinders,fuel,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
model,,,,,,,,,,,,,,,,,,,,,
"""t""",7310905114,https://eugene.craigslist.org/cto/d/eugene-191...,eugene,https://eugene.craigslist.org,27000,1913.0,ford,like new,4 cylinders,gas,...,rwd,compact,convertible,red,https://images.craigslist.org/00X0X_bYoeBEz0cA...,WOW! WOW! WOW!!! FIRE CHIEF CAR!!! One of a ...,or,44.068200,-123.081900,2021-04-22T09:50:49-0700
#NAME?,7311211732,https://fresno.craigslist.org/ctd/d/san-luis-o...,fresno / madera,https://fresno.craigslist.org,21925,2014.0,NaN,excellent,6 cylinders,gas,...,4wd,NaN,wagon,white,https://images.craigslist.org/00B0B_6b02RkNwR3...,----------------------------------------------...,ca,35.275769,-120.668013,2021-04-22T21:13:23-0700
#NAME?,7316526866,https://detroit.craigslist.org/wyn/cto/d/walle...,detroit metro,https://detroit.craigslist.org,6500,2002.0,chevrolet,good,6 cylinders,diesel,...,NaN,NaN,truck,NaN,https://images.craigslist.org/00C0C_iS2vxEWjKw...,"Two trucks for sale one is a 2002 c6500, the o...",mi,42.558200,-83.477300,2021-05-03T20:28:23-0400
#NAME?,7316746751,https://chicago.craigslist.org/nwc/ctd/d/hanov...,chicago,https://chicago.craigslist.org,27195,2017.0,NaN,excellent,6 cylinders,gas,...,fwd,NaN,mini-van,white,https://images.craigslist.org/00O0O_iOeppaeDtV...,Cars & Finance - All in One Place!Going extra ...,il,41.995781,-88.167287,2021-05-04T10:46:58-0500
#NAME?,7304427389,https://erie.craigslist.org/cto/d/erie-2010-me...,erie,https://erie.craigslist.org,10500,2010.0,mercedes-benz,excellent,6 cylinders,gas,...,4wd,NaN,sedan,grey,https://images.craigslist.org/00h0h_av7uAaphI1...,2010 Mercedes-Benz E350 4MATIC V6 3.5 L for sa...,pa,42.073800,-80.148400,2021-04-09T18:32:21-0400


In [25]:
df_4=df_2.copy()

In [15]:
df_4.loc[df_4['model']=='1500',['model','manufacturer']].value_counts()

model  manufacturer
1500   ram             3599
       chevrolet         97
       gmc               18
       dodge              4
dtype: int64

In [103]:
df_3.iloc[100:116]

,id,url,region,region_url,price,year,manufacturer,condition,cylinders,fuel,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
model,,,,,,,,,,,,,,,,,,,,,
.. ect.,7304172920,https://buffalo.craigslist.org/ctd/d/angola-ju...,buffalo,https://buffalo.craigslist.org,1234,2002.0,ford,NaN,NaN,other,...,NaN,NaN,NaN,NaN,https://images.craigslist.org/00Q0Q_5uDnk4SMpb...,Skip the Middle Man .. Get top dollar $$$ Car...,ny,42.636600,-79.049700,2021-04-09T11:33:21-0400
.. ect.,7316525434,https://buffalo.craigslist.org/ctd/d/angola-we...,buffalo,https://buffalo.craigslist.org,1234,2002.0,ford,NaN,NaN,other,...,NaN,NaN,NaN,NaN,https://images.craigslist.org/00Q0Q_5uDnk4SMpb...,Running or Not !!! Skip the Middle Man .. $$$...,ny,42.636600,-79.049700,2021-05-03T20:24:50-0400
.Cadillac XLR convertible,7316014113,https://denver.craigslist.org/cto/d/denver-cad...,denver,https://denver.craigslist.org,29000,2004.0,NaN,excellent,8 cylinders,gas,...,rwd,full-size,convertible,purple,https://images.craigslist.org/00C0C_4GdJWBeEbv...,2004 Cadillac XLR Neiman Marcus edition this c...,co,39.867200,-105.004100,2021-05-02T18:36:10-0600
/ accord,7315118598,https://providence.craigslist.org/ctd/d/provid...,rhode island,https://providence.craigslist.org,8500,2013.0,honda,excellent,6 cylinders,gas,...,fwd,compact,sedan,white,https://images.craigslist.org/00202_csgqtAR81U...,"2013 HONDA ACCORD WHITE 135K , LEATHER, V6 , S...",ri,41.797100,-71.425500,2021-04-30T20:36:13-0400
/ braun,7313030279,https://kpr.craigslist.org/cto/d/west-richland...,kennewick-pasco-richland,https://kpr.craigslist.org,7250,2003.0,dodge,fair,6 cylinders,gas,...,fwd,NaN,mini-van,blue,https://images.craigslist.org/00o0o_kfcDqC3K2x...,"2003 Dodge Caravan, Braun converted wheel chai...",wa,46.315300,-119.371400,2021-04-26T14:01:43-0700
/ durango sport,7315509536,https://richmond.craigslist.org/cto/d/richmond...,richmond,https://richmond.craigslist.org,20000,2015.0,dodge,good,6 cylinders,gas,...,4wd,full-size,SUV,grey,https://images.craigslist.org/00V0V_7ItxfTZ7Xu...,"2015 Dodge Durango 66,779 miles. Good condition",va,37.458000,-77.470000,2021-05-01T17:12:14-0400
/ vmi / ♿,7314403674,https://chicago.craigslist.org/sox/ctd/d/champ...,chicago,https://chicago.craigslist.org,10950,2006.0,chrysler,excellent,6 cylinders,gas,...,fwd,full-size,mini-van,white,https://images.craigslist.org/00c0c_8429zWAfT7...,* 2OO6 CHRYSLER TOWN n COUNTRY MINI VAN * V6 ...,il,40.107300,-88.278800,2021-04-29T12:33:35-0500
/ vmi ♿,7308270183,https://peoria.craigslist.org/cto/d/dewey-chry...,peoria,https://peoria.craigslist.org,10995,2006.0,chrysler,excellent,6 cylinders,gas,...,fwd,NaN,NaN,white,https://images.craigslist.org/00c0c_8429zWAfT7...,♿ CHRYSLER - VMI - WHEELCHAIR ACCESSIBLE MINI-...,il,40.313100,-88.277000,2021-04-17T10:54:30-0500
// vmi // ♿,7307904842,https://quincy.craigslist.org/cto/d/decatur-ch...,western IL,https://quincy.craigslist.org,10999,2006.0,chrysler,excellent,6 cylinders,gas,...,fwd,mid-size,mini-van,white,https://images.craigslist.org/00G0G_lFL2CGoeUQ...,♿ CHRYSLER - VMI - WHEELCHAIR ACCESSIBLE MINI-...,il,39.858300,-88.938200,2021-04-16T15:02:53-0500


In [26]:
df_4['cylinders'] = df_4.groupby('model')['cylinders'].ffill()

In [107]:
df_4.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     14147
model             4259
condition        19900
cylinders        38963
fuel              2312
odometer          2158
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [27]:
df_4['manufacturer'] = df_4.groupby('model')['manufacturer'].ffill()

In [113]:
df_4['manufacturer'].value_counts()

ford               62830
chevrolet          49130
toyota             31449
honda              19728
nissan             17207
jeep               17131
ram                15682
gmc                14712
bmw                13494
dodge              12046
mercedes-benz      10006
hyundai             9294
subaru              8914
volkswagen          8854
lexus               7651
kia                 7434
audi                6918
cadillac            6442
acura               5655
chrysler            5611
buick               5150
mazda               5035
infiniti            4435
lincoln             3968
volvo               3219
mitsubishi          3089
mini                2247
pontiac             2210
rover               1885
jaguar              1878
porsche             1163
mercury             1128
saturn              1069
alfa-romeo           856
tesla                768
fiat                 764
harley-davidson      138
datsun                63
land rover            11
aston-martin          11


In [114]:
df_2['manufacturer'].value_counts()

ford               62756
chevrolet          49071
toyota             31214
honda              19726
nissan             17206
jeep               17126
ram                15668
gmc                14704
bmw                13473
dodge              12036
mercedes-benz       9998
hyundai             9283
subaru              8913
volkswagen          8849
lexus               7651
kia                 7434
audi                6918
cadillac            6441
acura               5655
chrysler            5595
buick               5146
mazda               5008
infiniti            4434
lincoln             3967
volvo               3218
mitsubishi          3089
mini                2247
pontiac             2209
jaguar              1878
rover               1849
porsche             1161
mercury             1127
saturn              1069
alfa-romeo           856
tesla                768
fiat                 764
harley-davidson      138
datsun                59
land rover            11
aston-martin          11


In [115]:
df_4.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     13598
model             4259
condition        19900
cylinders        38963
fuel              2312
odometer          2158
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [16]:
df_4['fuel'].value_counts()

gas         323825
other        27117
diesel       23257
hybrid        4813
electric      1561
Name: fuel, dtype: int64

In [28]:
df_4['fuel']=df_4.groupby('model')['fuel'].ffill()

In [20]:
df_4.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     14147
model             4259
condition        19900
cylinders        36992
fuel              1475
odometer          2158
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [116]:
df_2.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     14147
model             4259
condition        19900
cylinders        85311
fuel              2312
odometer          2158
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [29]:
df_4['odometer'] = df_4['odometer'].fillna(df_4.groupby(['year','model'])['odometer'].transform('mean'))

In [30]:
df_4.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     13598
model             4259
condition        19900
cylinders        34798
fuel              1475
odometer           430
title_status      7259
transmission      1713
VIN             148688
drive           117275
size            275037
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [31]:
df_4['drive'].value_counts()

4wd    115132
fwd     96528
rwd     53950
Name: drive, dtype: int64

In [32]:
df_4.loc[df_4['manufacturer'] == 'subaru', ['drive']].value_counts()

drive
4wd      5012
rwd       123
fwd       113
dtype: int64

In [33]:
df_4['size'].value_counts()

full-size      56457
mid-size       31545
compact        17002
sub-compact     2844
Name: size, dtype: int64

In [34]:
df_4['size']= df_4.groupby('model')['size'].ffill()

In [35]:
df_4.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     13598
model             4259
condition        19900
cylinders        34798
fuel              1475
odometer           430
title_status      7259
transmission      1713
VIN             148688
drive           117275
size             95822
type             82787
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [36]:
df_4['type'].value_counts()

sedan          79230
SUV            69442
pickup         39761
truck          28714
other          19374
coupe          17503
hatchback      15815
wagon           9882
van             7695
convertible     7074
mini-van        4529
offroad          583
bus              496
Name: type, dtype: int64

In [38]:
df_4['type']= df_4.groupby('model')['type'].ffill()

In [39]:
df_4.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     13598
model             4259
condition        19900
cylinders        34798
fuel              1475
odometer           430
title_status      7259
transmission      1713
VIN             148688
drive           117275
size             95822
type             12457
paint_color     113531
image_url            0
description          2
state                0
lat               3399
long              3399
posting_date         0
dtype: int64

In [44]:
df_4.drop(['lat', 'long', 'image_url', 'VIN', 'posting_date'], axis=1, inplace=True)

In [48]:
df_4.drop('description', axis=1, inplace=True)

In [53]:
df_4.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year                 0
manufacturer     13598
model             4259
condition        19900
cylinders        34798
fuel              1475
odometer           430
title_status      7259
transmission      1713
drive           117275
size             95822
type             12457
paint_color     113531
state                0
dtype: int64

In [49]:
df_5=df_4.copy()

In [50]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382885 entries, 27 to 426879
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            382885 non-null  int64  
 1   url           382885 non-null  object 
 2   region        382885 non-null  object 
 3   region_url    382885 non-null  object 
 4   price         382885 non-null  int64  
 5   year          382885 non-null  float64
 6   manufacturer  369287 non-null  object 
 7   model         378626 non-null  object 
 8   condition     362985 non-null  object 
 9   cylinders     348087 non-null  object 
 10  fuel          381410 non-null  object 
 11  odometer      382455 non-null  float64
 12  title_status  375626 non-null  object 
 13  transmission  381172 non-null  object 
 14  drive         265610 non-null  object 
 15  size          287063 non-null  object 
 16  type          370428 non-null  object 
 17  paint_color   269354 non-null  object 
 18  sta

In [51]:
df_5.dropna(inplace=True) #164721
df_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164722 entries, 31 to 426872
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            164722 non-null  int64  
 1   url           164722 non-null  object 
 2   region        164722 non-null  object 
 3   region_url    164722 non-null  object 
 4   price         164722 non-null  int64  
 5   year          164722 non-null  float64
 6   manufacturer  164722 non-null  object 
 7   model         164722 non-null  object 
 8   condition     164722 non-null  object 
 9   cylinders     164722 non-null  object 
 10  fuel          164722 non-null  object 
 11  odometer      164722 non-null  float64
 12  title_status  164722 non-null  object 
 13  transmission  164722 non-null  object 
 14  drive         164722 non-null  object 
 15  size          164722 non-null  object 
 16  type          164722 non-null  object 
 17  paint_color   164722 non-null  object 
 18  sta

In [52]:
df_6=df_4.copy()

In [55]:
df_6.drop(['paint_color','drive'], axis=1, inplace=True)

In [56]:
df_6.dropna(inplace=True)
df_6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270520 entries, 31 to 426875
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            270520 non-null  int64  
 1   url           270520 non-null  object 
 2   region        270520 non-null  object 
 3   region_url    270520 non-null  object 
 4   price         270520 non-null  int64  
 5   year          270520 non-null  float64
 6   manufacturer  270520 non-null  object 
 7   model         270520 non-null  object 
 8   condition     270520 non-null  object 
 9   cylinders     270520 non-null  object 
 10  fuel          270520 non-null  object 
 11  odometer      270520 non-null  float64
 12  title_status  270520 non-null  object 
 13  transmission  270520 non-null  object 
 14  size          270520 non-null  object 
 15  type          270520 non-null  object 
 16  state         270520 non-null  object 
dtypes: float64(2), int64(2), object(13)
memory usag

In [57]:
df_4['region'].value_counts()

columbus               3363
jacksonville           3007
milwaukee              2905
grand rapids           2885
central NJ             2860
                       ... 
meridian                 20
southwest MS             12
kansas city               9
fort smith, AR            8
west virginia (old)       8
Name: region, Length: 404, dtype: int64